In [1]:
import os
import json
import duckdb
import pandas as pd

In [2]:
# Script pour générer des fichiers CSV à partir de données d'images et de segmentations

# Chemin vers le dossier racine 7272660
ROOT_DIR = os.getcwd() + '/dataset/7272660'

# Liste pour stocker les données
images_data = []
segmentations_data = []

# ID incrémentaux simulés
current_image_id = 1
current_segmentation_id = 1

# Les catégories et leurs sous-dossiers
CATEGORIES = {
    'Benign': 'Benign',
    'Malignant': 'Malignant',
    'Mormal': 'Normal'
}

# Parcours des catégories
for category_key, category_folder in CATEGORIES.items():
    category_path = os.path.join(ROOT_DIR, category_folder, category_folder)

    images_path = os.path.join(category_path, 'image')
    segmentation_path = os.path.join(category_path, 'segmentation')

    # Vérifier que le dossier existe
    if not os.path.exists(images_path):
        print(f"Dossier {images_path} manquant.")
        continue

    for file_name in os.listdir(images_path):
        if file_name.lower().endswith('.jpg'):
            # Préparer les infos pour la table image
            image_record = {
                'id_image': current_image_id,
                'file_name': file_name,
                'file_path': os.path.relpath(os.path.join(images_path, file_name), ROOT_DIR).replace("\\", "/"),
                'category': category_key
            }
            images_data.append(image_record)

            base_name = os.path.splitext(file_name)[0]

            # Pour chaque type de segmentation
            for seg_type in ['liver', 'mass', 'outline']:
                seg_folder = os.path.join(segmentation_path, seg_type)
                seg_file_path = os.path.join(seg_folder, base_name + '.json')

                if os.path.exists(seg_file_path):
                    # Lecture du contenu JSON
                    with open(seg_file_path, 'r', encoding='utf-8') as f:
                        annotation_content = json.load(f)

                    segmentation_record = {
                        'id_segmentation': current_segmentation_id,
                        'segmentation_type': seg_type.capitalize(),  # Liver, Mass, Outline
                        'annotation': json.dumps(annotation_content),  # pour l'insérer en base
                        'fk_id_image': current_image_id
                    }
                    segmentations_data.append(segmentation_record)
                    current_segmentation_id += 1
                else:
                    # Pas d'erreur si mass manque pour normal par exemple
                    if seg_type == 'mass' and category_key == 'normal':
                        continue
                    else:
                        print(f"Annotation {seg_type} manquante pour {file_name}")

            # Incrémenter l'image id
            current_image_id += 1

# Création des DataFrames
df_images = pd.DataFrame(images_data)
df_segmentations = pd.DataFrame(segmentations_data)

# Sauvegarde en CSV
output_images_csv = os.path.join(os.getcwd() + '/output_data/images.csv')
output_segmentations_csv = os.path.join(os.getcwd() + '/output_data/segmentations.csv')

df_images.to_csv(output_images_csv, index=False, encoding='utf-8')
df_segmentations.to_csv(output_segmentations_csv, index=False, encoding='utf-8')

print(f"CSV Images généré : {output_images_csv}")
print(f"CSV Segmentations généré : {output_segmentations_csv}")

Annotation liver manquante pour 229.jpg
Annotation liver manquante pour 306.jpg
Annotation liver manquante pour 374.jpg
Annotation mass manquante pour 1.jpg
Annotation mass manquante pour 10.jpg
Annotation mass manquante pour 100.jpg
Annotation mass manquante pour 11.jpg
Annotation mass manquante pour 12.jpg
Annotation mass manquante pour 13.jpg
Annotation mass manquante pour 14.jpg
Annotation mass manquante pour 15.jpg
Annotation mass manquante pour 16.jpg
Annotation mass manquante pour 17.jpg
Annotation mass manquante pour 18.jpg
Annotation mass manquante pour 19.jpg
Annotation mass manquante pour 2.jpg
Annotation mass manquante pour 20.jpg
Annotation mass manquante pour 21.jpg
Annotation mass manquante pour 22.jpg
Annotation mass manquante pour 23.jpg
Annotation mass manquante pour 24.jpg
Annotation mass manquante pour 25.jpg
Annotation mass manquante pour 26.jpg
Annotation mass manquante pour 27.jpg
Annotation mass manquante pour 28.jpg
Annotation mass manquante pour 29.jpg
Annotat

In [3]:
connection_database = duckdb.connect(database='annotated_ultrasound.db', read_only=False)

In [4]:
dataframe_images = pd.read_csv(output_images_csv)
dataframe_segmentations = pd.read_csv(output_segmentations_csv)

In [5]:
connection_database.register('images_dataframe', dataframe_images)
connection_database.register('segmentations_dataframe', dataframe_segmentations)

In [6]:
connection_database.execute("CREATE TABLE t_image AS SELECT * FROM images_dataframe")
connection_database.execute("CREATE TABLE t_segmentation AS SELECT * FROM segmentations_dataframe")